## K-Means Clustering

In [5]:
import pandas as pd
dcssd = pd.read_csv('Data.csv')
dcssd.rename({'cluster number':'cluster_number'}, axis=1, inplace=True)
dcssd.to_csv('Data.csv')
dwq = pd.read_csv('Data_modified.csv')
dwq.rename({'cluster number':'cluster_number'}, axis=1, inplace=True)
dwq.to_csv('Data_modified.csv')

In [19]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

#read from One-Hot Encoded Data file 

X = pd.read_csv('Data_modified.csv')
X = X.iloc[:, 1:]
kmeans = KMeans(n_clusters=100, random_state=0).fit(X)

#K-Means Clustering and get 100 centroids for ~7000 data samples, i.e 100 clusters

centroids = kmeans.cluster_centers_
centroid_df = pd.DataFrame(centroids, columns=X.columns)

def roundoff(x):
    return int(round(x, 0))


cdn = centroid_df[['tenure', 'MonthlyCharges', 'TotalCharges']]

decode_cols = ['PaymentMethod', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
               'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract']
dic_decode = {}

cdc = centroid_df.drop(['tenure', 'MonthlyCharges', 'TotalCharges',
                        'gender','SeniorCitizen','Partner','Dependents','PhoneService',
                        'PaperlessBilling','Churn'], axis=1)

cdm = centroid_df[['gender','SeniorCitizen','Partner','Dependents','PhoneService','PaperlessBilling','Churn']]
cdm = cdm.applymap(roundoff)


for x in decode_cols:
    l_dec = []
    for y in cdc.columns:
        if (str(x) in str(y) and str(x)!=str(y)):
            l_dec.append(y)
        dic_decode[x] = l_dec
        
cdc = cdc.join(cdm)
  #.applymap(roundoff)

for x in dic_decode.keys():
    cdc[x] = pd.Series(['dummy' for i in range(len(cdc.index))])

for i in range(len(cdc.index)):
    for x in dic_decode.keys():
        t1 = dic_decode[x][0]
        for y in dic_decode[x]:
            if(cdc.loc[i,y]>=cdc.loc[i,t1]):
                t1 = y
                cdc.loc[i,x] = str(t1).split('#')[1]


cdf = cdn.join(cdc)
for x in dic_decode.keys():
    cdf.drop(columns=dic_decode[x], inplace=True)

X['cluster_number'] = kmeans.labels_
#X.to_csv('Data_modified.csv')

In [20]:
pd.set_option('max_columns',60)
cdf

,tenure,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,PhoneService,PaperlessBilling,Churn,PaymentMethod,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,5.013158,81.048684,395.541447,0,0,0,0,1,1,1,Electronic check,No,Fiber optic,No,No,No,No,No,No,Month-to-month
1,57.272727,89.792727,5024.608182,0,0,1,0,1,1,0,Credit card (automatic),Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year
2,30.385542,77.511446,2199.992771,1,0,1,0,1,1,0,Electronic check,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month
3,67.394737,105.643421,7127.030263,0,0,1,0,1,1,0,Credit card (automatic),Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year
4,47.301887,81.031132,3647.137736,1,0,0,0,1,1,0,Electronic check,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,39.596491,70.066667,2567.506140,1,0,1,0,1,1,0,Bank transfer (automatic),No,DSL,No,No,No,No,No,No,Month-to-month
96,41.833333,80.091667,3176.265741,1,0,1,0,1,1,0,Electronic check,Yes,Fiber optic,No,Yes,No,No,No,Yes,Month-to-month
97,66.756757,95.054054,6313.243243,1,0,1,0,1,0,0,Electronic check,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year
98,7.457364,26.737984,176.760853,1,0,0,0,1,0,0,Mailed check,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month


In [21]:
#write cluster number for each record to original data 

data = pd.read_csv('Data.csv')
empty_cols=['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
           'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
           'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport',
           'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
           'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
for i in empty_cols:
    data[i]=data[i].replace(" ",np.nan)
data = data.dropna()
data['cluster_number'] = kmeans.labels_
data.to_csv('Data.csv')

#decode the one-hot encoded data in the centroids to get each Centroid as a data sample

cdf['gender'].replace({1: "Male", 0: "Female"}, inplace=True)

binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for i in binary_cols:
    cdf[i] = cdf[i].replace({1: "Yes", 0: "No"})



In [26]:
pd.set_option('max_columns',60)
cdf

,tenure,MonthlyCharges,TotalCharges,gender,SeniorCitizen,Partner,Dependents,PhoneService,PaperlessBilling,Churn,PaymentMethod,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract
0,5.013158,81.048684,395.541447,Female,0,No,No,Yes,Yes,1,Electronic check,No,Fiber optic,No,No,No,No,No,No,Month-to-month
1,57.272727,89.792727,5024.608182,Female,0,Yes,No,Yes,Yes,0,Credit card (automatic),Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year
2,30.385542,77.511446,2199.992771,Male,0,Yes,No,Yes,Yes,0,Electronic check,No,Fiber optic,No,No,No,No,Yes,Yes,Month-to-month
3,67.394737,105.643421,7127.030263,Female,0,Yes,No,Yes,Yes,0,Credit card (automatic),Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year
4,47.301887,81.031132,3647.137736,Male,0,No,No,Yes,Yes,0,Electronic check,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,39.596491,70.066667,2567.506140,Male,0,Yes,No,Yes,Yes,0,Bank transfer (automatic),No,DSL,No,No,No,No,No,No,Month-to-month
96,41.833333,80.091667,3176.265741,Male,0,Yes,No,Yes,Yes,0,Electronic check,Yes,Fiber optic,No,Yes,No,No,No,Yes,Month-to-month
97,66.756757,95.054054,6313.243243,Male,0,Yes,No,Yes,No,0,Electronic check,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year
98,7.457364,26.737984,176.760853,Male,0,No,No,Yes,No,0,Mailed check,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month


In [31]:
cdf = cdf[['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn']]
cdf['cluster_number'] = cdf.index
cdf.set_index('cluster_number',inplace=True)
cdf.to_csv('Data_centroids.csv', index=False)

In [7]:
cdf1=pd.read_csv('Data_centroids.csv')
cdf1.rename({'cluster number':'cluster_number'}, axis=1, inplace=True)

cdf1.to_csv('Data_centroids.csv', index=False)